In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import pandas as pd


import tensorflow as tf

# 设置GPU设备
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU set successfully!")
    except RuntimeError as e:
        print(e)
    

GPU set successfully!


使用gpu训练，看看速度。 tensorflow2.6.0 keras2.6.0  CUDAv12.8 cudnn-windows-x86_64-8.9.7.29_cuda12-archive

In [3]:
orig_data = pd.read_csv("data_set/从11个特征预测学生的测试分数.csv")
X , Y = orig_data.iloc[:,0:10] , orig_data.iloc[:,-1] #切片
X = X.drop(columns=["school","classroom","student_id"],axis=1)#按列丢弃无关项
X = pd.get_dummies(X,columns=["school_setting","school_type","teaching_method","gender","lunch"],dtype=int)#独热编码 #orig_data.iloc[0:5,] X.iloc[0:5,]
X

,n_student,pretest,school_setting_Rural,school_setting_Suburban,school_setting_Urban,school_type_Non-public,school_type_Public,teaching_method_Experimental,teaching_method_Standard,gender_Female,gender_Male,lunch_Does not qualify,lunch_Qualifies for reduced/free lunch
0,20.0,62.0,0,0,1,1,0,0,1,1,0,1,0
1,20.0,66.0,0,0,1,1,0,0,1,1,0,1,0
2,20.0,64.0,0,0,1,1,0,0,1,0,1,1,0
3,20.0,61.0,0,0,1,1,0,0,1,1,0,1,0
4,20.0,64.0,0,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,30.0,39.0,0,0,1,0,1,0,1,1,0,1,0
2129,30.0,38.0,0,0,1,0,1,0,1,1,0,0,1
2130,30.0,45.0,0,0,1,0,1,0,1,1,0,0,1
2131,30.0,46.0,0,0,1,0,1,0,1,0,1,0,1


一层神经网络模型  
激活函数f(x)= x  
损失函数均方误差  
优化器Adam(默认比较优秀的优化器 结合了自适应梯度Adagrad和RMSProp防梯度急剧下降的优点)

In [4]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)#Sequential模型结构 多个网络层且无多余分支的堆叠 Dense就是常用的全连接层 运算就是output = activation(dot(input, kernel)+bias) units为该层的输出维度 而非输入维度  
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)) #adam（前期降低损失迅速，随迭代次数衰减学习率）学习率提高一点 
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) #整个数据一共遍历epoch遍 每一次拿batch_size个样本来算损失 所以一个epoch需要算epoch/batch_size遍 更新同样次数的参数
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 979us/step - loss: 2543.8308
Epoch 2/100
54/54 [==============================] - 0s 1ms/step - loss: 345.0066
Epoch 3/100
54/54 [==============================] - 0s 826us/step - loss: 52.8748
Epoch 4/100
54/54 [==============================] - 0s 903us/step - loss: 37.7791
Epoch 5/100
54/54 [==============================] - 0s 1ms/step - loss: 35.1561
Epoch 6/100
54/54 [==============================] - 0s 1ms/step - loss: 32.6324
Epoch 7/100
54/54 [==============================] - 0s 1ms/step - loss: 30.1926
Epoch 8/100
54/54 [==============================] - 0s 983us/step - loss: 27.8799
Epoch 9/100
54/54 [==============================] - 0s 979us/step - loss: 25.7364
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss: 23.8201 - val_loss: 26.1615
Epoch 11/100
54/54 [==============================] - 0s 773us/step - loss: 22.0880
Epoch 12/100
54/54 [==============================] - 0s 841us/step - lo

换个优化器 随机梯度下降

In [5]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.SGD(learning_rate=0.01,clipvalue=0.5))#梯度裁剪 Gradient Clipping +-0.5
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 859us/step - loss: 2203.6917
Epoch 2/100
54/54 [==============================] - 0s 785us/step - loss: 601.6945
Epoch 3/100
54/54 [==============================] - 0s 879us/step - loss: 42.1208
Epoch 4/100
54/54 [==============================] - 0s 781us/step - loss: 19.1598
Epoch 5/100
54/54 [==============================] - 0s 803us/step - loss: 17.7152
Epoch 6/100
54/54 [==============================] - 0s 745us/step - loss: 16.2958
Epoch 7/100
54/54 [==============================] - 0s 810us/step - loss: 15.2241
Epoch 8/100
54/54 [==============================] - 0s 786us/step - loss: 14.4278
Epoch 9/100
54/54 [==============================] - 0s 925us/step - loss: 13.8414
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 13.3639 - val_loss: 12.6784
Epoch 11/100
54/54 [==============================] - 0s 905us/step - loss: 12.9375
Epoch 12/100
54/54 [==============================] - 0s 852us/s

自适应梯度 (对初始位置要求很严格 不然直接3000遍历还有1000损失 下降平缓均匀

In [6]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adagrad(learning_rate=0.01,epsilon=1e-6))
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 795us/step - loss: 1656.7402
Epoch 2/100
54/54 [==============================] - 0s 809us/step - loss: 1177.4430
Epoch 3/100
54/54 [==============================] - 0s 791us/step - loss: 927.8684
Epoch 4/100
54/54 [==============================] - 0s 906us/step - loss: 757.2111
Epoch 5/100
54/54 [==============================] - 0s 881us/step - loss: 630.1652
Epoch 6/100
54/54 [==============================] - 0s 804us/step - loss: 531.1011
Epoch 7/100
54/54 [==============================] - 0s 914us/step - loss: 452.0047
Epoch 8/100
54/54 [==============================] - 0s 1ms/step - loss: 387.5287
Epoch 9/100
54/54 [==============================] - 0s 797us/step - loss: 334.1453
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 289.6642 - val_loss: 213.3415
Epoch 11/100
54/54 [==============================] - 0s 770us/step - loss: 252.3220
Epoch 12/100
54/54 [==============================] - 0

结合梯度平方的指数移动平均数来调节学习率的变化。能够在不稳定（Non-Stationary）的目标函数情况下进行很好地收敛。递归神经网络时的一个良好选择？

In [7]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9, epsilon=1e-6)) #rho衰减系数 epsilon数值稳定性的小常数？
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 909us/step - loss: 6842.5713
Epoch 2/100
54/54 [==============================] - 0s 1ms/step - loss: 1720.3939
Epoch 3/100
54/54 [==============================] - 0s 941us/step - loss: 134.9007
Epoch 4/100
54/54 [==============================] - 0s 1ms/step - loss: 30.0543
Epoch 5/100
54/54 [==============================] - 0s 910us/step - loss: 18.1644
Epoch 6/100
54/54 [==============================] - 0s 943us/step - loss: 15.1702
Epoch 7/100
54/54 [==============================] - 0s 1ms/step - loss: 13.7345
Epoch 8/100
54/54 [==============================] - 0s 915us/step - loss: 12.8666
Epoch 9/100
54/54 [==============================] - 0s 1ms/step - loss: 12.3767
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 12.1782 - val_loss: 11.1597
Epoch 11/100
54/54 [==============================] - 0s 1ms/step - loss: 11.8809
Epoch 12/100
54/54 [==============================] - 0s 916us/step - lo